In [22]:
import pandas as pd
import requests
import json

In [23]:
#Advantage api configuration
base_url = 'https://www.alphavantage.co/query'
token = "OYDV8S8KFCSH3B35"

In [24]:
#Parameters for stocks on market data
function = 'TIME_SERIES_INTRADAY'
symbol= 'AAPL'
interval = '15min'
adjusted=True
month='2023-01'
size = 'full'

In [25]:
parameters_market_data = {'function':function, 'symbol':symbol, 'interval':interval, 
            'adjusted':adjusted,'outputsize':size,'apikey':token, 'month':month, }
r = requests.get(base_url,params=parameters_market_data)
r.json()

{'Meta Data': {'1. Information': 'Intraday (15min) open, high, low, close prices and volume',
  '2. Symbol': 'AAPL',
  '3. Last Refreshed': '2023-01-31 19:45:00',
  '4. Interval': '15min',
  '5. Output Size': 'Full size',
  '6. Time Zone': 'US/Eastern'},
 'Time Series (15min)': {'2023-01-31 19:45:00': {'1. open': '143.3610',
   '2. high': '143.3840',
   '3. low': '143.2180',
   '4. close': '143.3820',
   '5. volume': '13462'},
  '2023-01-31 19:30:00': {'1. open': '143.2910',
   '2. high': '143.3340',
   '3. low': '143.2480',
   '4. close': '143.3320',
   '5. volume': '9052'},
  '2023-01-31 19:15:00': {'1. open': '143.4110',
   '2. high': '143.4340',
   '3. low': '143.2780',
   '4. close': '143.3220',
   '5. volume': '10172'},
  '2023-01-31 19:00:00': {'1. open': '143.3910',
   '2. high': '143.5240',
   '3. low': '143.3870',
   '4. close': '143.4120',
   '5. volume': '6275'},
  '2023-01-31 18:45:00': {'1. open': '143.4010',
   '2. high': '143.4640',
   '3. low': '143.3580',
   '4. close

In [26]:
data = r.json()['Time Series (15min)']
data_df = pd.DataFrame.from_dict(data, orient="index")
data_df

,1. open,2. high,3. low,4. close,5. volume
2023-01-31 19:45:00,143.3610,143.3840,143.2180,143.3820,13462
2023-01-31 19:30:00,143.2910,143.3340,143.2480,143.3320,9052
2023-01-31 19:15:00,143.4110,143.4340,143.2780,143.3220,10172
2023-01-31 19:00:00,143.3910,143.5240,143.3870,143.4120,6275
2023-01-31 18:45:00,143.4010,143.4640,143.3580,143.3920,4301
...,...,...,...,...,...
2023-01-03 05:00:00,131.0170,131.0200,130.6250,130.7490,7698
2023-01-03 04:45:00,130.8380,130.9200,130.7850,130.9180,14892
2023-01-03 04:30:00,130.5380,130.8600,130.5150,130.7690,12334
2023-01-03 04:15:00,130.8280,130.9400,130.5150,130.5190,25653


In [27]:
# Prameters for Market News and Sentinment

function_sentiment='NEWS_SENTIMENT'
tickers='AAPL'
topics='technology, manufacturing, retail_wholesale, financial_markets'
time_from='20230101T0130'
time_to='20230131T0130'
limit=50


In [30]:
parameters_news_sentiment_data = {'function':function_sentiment, 'tickers':tickers, 'topics':topics, 
            'time_from':time_from,'time_to':time_to,'apikey':token }
r = requests.get(base_url,params=parameters_news_sentiment_data)
r.json()

{'items': '50',
 'sentiment_score_definition': 'x <= -0.35: Bearish; -0.35 < x <= -0.15: Somewhat-Bearish; -0.15 < x < 0.15: Neutral; 0.15 <= x < 0.35: Somewhat_Bullish; x >= 0.35: Bullish',
 'relevance_score_definition': '0 < x <= 1, with a higher score indicating higher relevance.',
 'feed': [{'title': 'Tech, megacaps drag Wall Street to lower close as big market week kicks off',
   'url': 'https://www.moneycontrol.com/news/business/markets/tech-megacaps-drag-wall-street-to-lower-close-as-big-market-week-kicks-off-9968011.html',
   'time_published': '20230131T010815',
   'authors': [],
   'summary': 'The heavyweight tech sector ( .SPLRCT ) dropped 1.9% while energy ( .SPNY ) shed 2.3%, the biggest drop among the SP 500 sectors. Shares of Apple Inc ( AAPL.O ) , Amazon.com Inc ( AMZN.O ) and Google parent Alphabet Inc ( GOOGL.O ) , which are due to post results later this week, all slumped.',
   'banner_image': 'https://images.moneycontrol.com/static-mcnews/2017/04/wall_street1.jpg?imp

In [39]:
data = r.json()
data_feed = data['feed']

In [37]:
data['feed'][0]['ticker_sentiment']

[{'ticker': 'GOOG',
  'relevance_score': '0.279015',
  'ticker_sentiment_score': '-0.036345',
  'ticker_sentiment_label': 'Neutral'},
 {'ticker': 'WFC',
  'relevance_score': '0.141726',
  'ticker_sentiment_score': '0.071045',
  'ticker_sentiment_label': 'Neutral'},
 {'ticker': 'AAPL',
  'relevance_score': '0.141726',
  'ticker_sentiment_score': '-0.022032',
  'ticker_sentiment_label': 'Neutral'}]

In [ ]:
topics='technology, manufacturing, retail_wholesale, financial_markets'
time_from='20230101T0130'
time_to='20230131T0130'

In [ ]:
def getSentiment(symbol:str, function_sentiment:str, topics:str, time_from:str, time_to:str):
    url='https://finnhub.io/api/v1/news-sentiment'
    parameters_news_sentiment_data = {'function':function_sentiment, 'tickers':symbol, 'topics':topics, 
            'time_from':time_from,'time_to':time_to,'apikey':token }
    r = requests.get(base_url,params=parameters_news_sentiment_data)
    data = r.json()
    data_feed = data['feed']
    data_sentiment = []
    for i in data_feed:
        for item in i['ticker_sentiment']:
            if item['ticker'] == symbol:
                data_sentiment.append([i['time_published'],i['source_domain'],item['relevance_score'],item['ticker_sentiment_label']])
    return data_sentiment

In [56]:
import json
from typing import Union, List

def getSentiment(
    symbol: str,
    function_sentiment: str,
    topics: Union[str, List[str]],
    time_from: Union[str, int],
    time_to: Union[str, int]
):
    url = 'https://finnhub.io/api/v1/news-sentiment'
    
    # Convert topics to a comma-separated string if it is a list of strings
    if isinstance(topics, list):
        topics = ','.join(topics)
    
    # Convert time_from and time_to to strings if they are integers
    time_from = str(time_from)
    time_to = str(time_to)
    
    parameters_news_sentiment_data = {
        'function': function_sentiment,
        'tickers': symbol,
        'topics': topics,
        'time_from': time_from,
        'time_to': time_to,
        'apikey': token
    }
    r = requests.get(base_url, params=parameters_news_sentiment_data)
    data = r.json()
    data_feed = data['feed']
    data_sentiment = []
    for i in data_feed:
        for item in i['ticker_sentiment']:
            if item['ticker'] == symbol:
                data_sentiment.append({
                    'time_published': i['time_published'],
                    'source_domain': i['source_domain'],
                    'relevance_score': item['relevance_score'],
                    'ticker_sentiment_label': item['ticker_sentiment_label']
                })
    return data_sentiment

# Example usage
topics_str = "technology, manufacturing, retail_wholesale, financial_markets"
result = getSentiment("AAPL", "NEWS_SENTIMENT", topics_str, time_from='20230101T0130', time_to='20230131T0130')

# Write the result to a JSON file
with open('result.json', 'w') as json_file:
    json.dump(result, json_file) 


In [59]:
result_list = getSentiment(
    symbol='AAPL', function_sentiment='NEWS_SENTIMENT', topics='technology, manufacturing, retail_wholesale, financial_markets', time_from='20230101T0130', time_to='20230131T0130')
                

In [60]:
result_list

[{'time_published': '20230131T010815',
  'source_domain': 'www.moneycontrol.com',
  'relevance_score': '0.141726',
  'ticker_sentiment_label': 'Neutral'},
 {'time_published': '20230130T234200',
  'source_domain': 'www.marketwatch.com',
  'relevance_score': '0.801457',
  'ticker_sentiment_label': 'Bearish'},
 {'time_published': '20230130T224800',
  'source_domain': 'www.zacks.com',
  'relevance_score': '0.664',
  'ticker_sentiment_label': 'Bullish'},
 {'time_published': '20230130T215500',
  'source_domain': 'www.investors.com',
  'relevance_score': '0.246582',
  'ticker_sentiment_label': 'Neutral'},
 {'time_published': '20230130T214700',
  'source_domain': 'www.reuters.com',
  'relevance_score': '0.094762',
  'ticker_sentiment_label': 'Neutral'},
 {'time_published': '20230130T211456',
  'source_domain': 'www.kiplinger.com',
  'relevance_score': '0.145583',
  'ticker_sentiment_label': 'Neutral'},
 {'time_published': '20230130T204600',
  'source_domain': 'www.marketwatch.com',
  'relevanc